<a href="https://colab.research.google.com/github/mohan22iitk/Chirp-Bird-Song-Recognition/blob/main/Mohan_220656.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'urbansound8k:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F500970%2F928025%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240717%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240717T132312Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Daf39e07a672652a68e6836ff4a67f9b4686f533c4845e414ee7049b813df970508488e83ae86dc304f67d5d04079530ce352deb92d89bb891eabe8893297084d145644cb844f98a94b0902600632dd3b0435ff44929f95084ce8d9cde8e4fb050f2ec01fa4ce7bd7bfecd6285d85a289be00161cd03ca321107fb58ebe55ddc5ab8721b9bd1a51a1f5c64b2979f629a11a9344dcd2992844fa08f172eb572d300bb00e0c641c364b1f28ef4851052b139ce269ab529be304b29dc002f9a837dc1295d3c438a0d944f20bfdc06fb6e6e73034150a92d6588e0d5a429a1ac28d78fa5fb3473116735495637ba64fa0fee77d5db90c4382e24ae8b3c8407e891165'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 6026232524 bytes downloaded
Downloaded and uncompressed: urbansound8k
Data source import complete.


In [3]:

# Importing libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import IPython.display as ipd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import os
import librosa
import librosa.display


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


Streaming output truncated to the last 5000 lines.
/kaggle/input/urbansound8k/fold10/77927-3-1-1.wav
/kaggle/input/urbansound8k/fold10/119067-0-0-1.wav
/kaggle/input/urbansound8k/fold10/196084-2-0-2.wav
/kaggle/input/urbansound8k/fold10/188813-7-1-0.wav
/kaggle/input/urbansound8k/fold10/102857-5-0-18.wav
/kaggle/input/urbansound8k/fold10/83502-0-0-3.wav
/kaggle/input/urbansound8k/fold10/93567-8-0-15.wav
/kaggle/input/urbansound8k/fold10/74364-8-1-12.wav
/kaggle/input/urbansound8k/fold10/162134-7-8-3.wav
/kaggle/input/urbansound8k/fold10/188813-7-11-0.wav
/kaggle/input/urbansound8k/fold10/72220-3-0-0.wav
/kaggle/input/urbansound8k/fold10/28808-1-0-1.wav
/kaggle/input/urbansound8k/fold10/181624-4-5-0.wav
/kaggle/input/urbansound8k/fold10/93567-8-0-14.wav
/kaggle/input/urbansound8k/fold10/197554-2-0-82.wav
/kaggle/input/urbansound8k/fold10/15544-5-0-7.wav
/kaggle/input/urbansound8k/fold10/165166-8-0-1.wav
/kaggle/input/urbansound8k/fold10/189985-0-0-6.wav
/kaggle/input/urbansound8k/fold10

In [4]:
df = pd.read_csv("/kaggle/input/urbansound8k/UrbanSound8K.csv")

In [5]:
df.head()

,slice_file_name,fsID,start,end,salience,fold,classID,class
0,100032-3-0-0.wav,100032,0.0,0.317551,1,5,3,dog_bark
1,100263-2-0-117.wav,100263,58.5,62.500000,1,5,2,children_playing
2,100263-2-0-121.wav,100263,60.5,64.500000,1,5,2,children_playing
3,100263-2-0-126.wav,100263,63.0,67.000000,1,5,2,children_playing
4,100263-2-0-137.wav,100263,68.5,72.500000,1,5,2,children_playing


In [6]:
df['class'].unique()

array(['dog_bark', 'children_playing', 'car_horn', 'air_conditioner',
       'street_music', 'gun_shot', 'siren', 'engine_idling', 'jackhammer',
       'drilling'], dtype=object)

In [7]:
df['slice_file_name'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 8732 entries, 0 to 8731
Series name: slice_file_name
Non-Null Count  Dtype 
--------------  ----- 
8732 non-null   object
dtypes: object(1)
memory usage: 68.3+ KB


In [8]:
# Selecting 5 classes from the above 10 categories in the dataset
selected_classes = ['car_horn', 'siren', 'dog_bark', 'jackhammer', 'drilling']

# Creating new dataframe with only the rows that have selected classes
new_df = df[df['class'].isin(selected_classes)]

new_df

,slice_file_name,fsID,start,end,salience,fold,classID,class
0,100032-3-0-0.wav,100032,0.000000,0.317551,1,5,3,dog_bark
9,100648-1-0-0.wav,100648,4.823402,5.471927,2,10,1,car_horn
10,100648-1-1-0.wav,100648,8.998279,10.052132,2,10,1,car_horn
11,100648-1-2-0.wav,100648,16.699509,17.104837,2,10,1,car_horn
12,100648-1-3-0.wav,100648,17.631764,19.253075,2,10,1,car_horn
...,...,...,...,...,...,...,...,...
8727,99812-1-2-0.wav,99812,159.522205,163.522205,2,7,1,car_horn
8728,99812-1-3-0.wav,99812,181.142431,183.284976,2,7,1,car_horn
8729,99812-1-4-0.wav,99812,242.691902,246.197885,2,7,1,car_horn
8730,99812-1-5-0.wav,99812,253.209850,255.741948,2,7,1,car_horn


In [9]:
new_df['class'].unique()

array(['dog_bark', 'car_horn', 'siren', 'jackhammer', 'drilling'],
      dtype=object)

In [10]:
y1,sr1 = librosa.load("/kaggle/input/urbansound8k/fold10/100648-1-3-0.wav")

In [11]:
ipd.Audio(y1,rate = sr1)

In [12]:
y2,sr2 = librosa.load("/kaggle/input/urbansound8k/fold5/100032-3-0-0.wav")

In [13]:
ipd.Audio(y2,rate = sr2)

In [14]:
mfccs = librosa.feature.mfcc(y=y1, sr=sr1, n_mfcc = 40)
mfccs

array([[-2.8385641e+02, -1.9600438e+02, -1.7693327e+02, ...,
        -1.9837999e+02, -1.9646992e+02, -1.8086203e+02],
       [ 7.1941719e+01,  6.2967499e+01,  6.3393646e+01, ...,
         1.1242232e+02,  1.0714292e+02,  9.6308960e+01],
       [-3.0946215e+01, -2.6726728e+01, -2.1977465e+01, ...,
        -9.2372255e+00, -1.0602974e+01, -9.1316719e+00],
       ...,
       [-5.4660912e+00, -3.6881995e-01, -1.1930857e+00, ...,
        -3.5530939e+00, -6.7237086e+00, -6.9404321e+00],
       [-2.1268654e-01,  1.7802581e+00, -2.3972518e+00, ...,
        -5.0999398e+00, -6.7743081e-01,  3.2616076e+00],
       [ 4.6264496e+00,  4.8848820e+00,  3.3231390e+00, ...,
         3.2673860e+00,  7.0808659e+00,  1.2326031e+01]], dtype=float32)

In [15]:
print(mfccs.shape)

(40, 70)


In [16]:
def extract_features(file_name):
    audio, sample_rate = librosa.load(file_name)
    mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    # Taking average in time frames
    mfccs_processed = np.mean(mfccs.T,axis=0)

    return mfccs_processed

In [17]:
    features = []
    # Iterate through each sound file and extract the features
    for index, row in new_df.iterrows():
        file_name = os.path.join(os.path.abspath("/kaggle/input/urbansound8k/"),'fold'+str(row["fold"])+'/',str(row["slice_file_name"]))
        label = row["class"]
        data = extract_features(file_name)

        features.append([data,label])
    # Convert into dataframe
    featuresdf = pd.DataFrame(features, columns=['feature','label'])

/usr/local/lib/python3.10/dist-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1323
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1103
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1523
  warnings.warn(


In [18]:
featuresdf

,feature,label
0,"[-211.93698, 62.58121, -122.813156, -60.745293...",dog_bark
1,"[-188.62564, 102.429115, -2.3306005, -10.94615...",car_horn
2,"[-195.3337, 99.1228, -16.376387, 2.7363458, -1...",car_horn
3,"[-200.00314, 79.35082, -13.392574, 17.59861, -...",car_horn
4,"[-196.05116, 90.81921, -17.248508, 14.91505, -...",car_horn
...,...,...
4353,"[-391.3721, 125.36239, -40.634056, 25.899754, ...",car_horn
4354,"[-339.42715, 76.409096, -35.30988, 41.984035, ...",car_horn
4355,"[-297.31534, 102.090256, -36.75108, 26.816488,...",car_horn
4356,"[-336.92386, 115.3139, -44.84709, 24.968985, -...",car_horn


In [19]:
featuresdf.feature[0].shape

(40,)

In [20]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


# Convert features and labels into PyTorch tensors
X = torch.tensor(featuresdf['feature'].tolist(), dtype=torch.float32)
y = featuresdf['label']

# Encode the classification labels using LabelEncoder
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Convert the encoded labels to one-hot encoded format
yy = F.one_hot(torch.tensor(y_encoded), num_classes=len(le.classes_))

# Split the data into training and testing sets if needed
# For example, using sklearn's train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, yy, test_size=0.2, random_state=127)

# Create DataLoader for training and testing sets
batch_size = 64
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


<ipython-input-20-5a175d8d4ac7>:11: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  X = torch.tensor(featuresdf['feature'].tolist(), dtype=torch.float32)


In [21]:
le.classes_

array(['car_horn', 'dog_bark', 'drilling', 'jackhammer', 'siren'],
      dtype=object)

In [22]:
np.unique(y_encoded)

array([0, 1, 2, 3, 4])

In [23]:
x_train_tensor = torch.tensor(np.array(X_train), dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
x_test_tensor = torch.tensor(np.array(X_test), dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

<ipython-input-23-61daa2899b7c>:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
<ipython-input-23-61daa2899b7c>:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_test_tensor = torch.tensor(y_test, dtype=torch.float32)


In [24]:
# Define the neural network model
class SimpleFeedforwardNN(nn.Module):
    def __init__(self, input_size, num_labels):
        super(SimpleFeedforwardNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 256)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(256, 256)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(0.5)
        self.fc3 = nn.Linear(256, num_labels)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.dropout2(x)
        x = self.fc3(x)
        x = self.softmax(x)
        return x

# Create an instance of the model
input_size = 40  # Assuming 40 features
num_labels = yy.shape[1]  # Assuming you have defined yy somewhere
model = SimpleFeedforwardNN(input_size, num_labels)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 100  # You can adjust this
for epoch in range(num_epochs):
    model.train()
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, torch.argmax(batch_y, dim=1))
        loss.backward()
        optimizer.step()

# Evaluate the model on the test set
model.eval()
with torch.no_grad():
    outputs = model(x_train_tensor)
    _, predicted = torch.max(outputs, 1)
    correct_train = (predicted == torch.argmax(y_train_tensor, dim=1)).sum().item()
    total_train = y_train_tensor.size(0)
    train_accuracy = correct_train / total_train * 100

with torch.no_grad():
    outputs = model(x_test_tensor)
    _, predicted = torch.max(outputs, 1)
    correct_test = (predicted == torch.argmax(y_test_tensor, dim=1)).sum().item()
    total_test = y_test_tensor.size(0)
    test_accuracy = correct_test / total_test * 100

print(f'Train Accuracy: {train_accuracy:.2f}%')
print(f'Test Accuracy: {test_accuracy:.2f}%')


Train Accuracy: 94.21%
Test Accuracy: 92.43%


In [25]:
audiofile = "/kaggle/input/urbansound8k/fold5/100032-3-0-0.wav"
feature = extract_features(audiofile)

In [26]:
feature=torch.tensor(feature.tolist(), dtype=torch.float32)
feature.shape

torch.Size([40])

In [27]:
# Add an extra dimension to simulate a batch of size 1
feature = feature.unsqueeze(0)
model.eval()
with torch.no_grad():
    outputs = model(feature)
    _, predicted = torch.max(outputs, 1)

print("Predictions:", predicted)

Predictions: tensor([1])
